In [1]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
from pyspark_helpers import *
sc, spark, conf = initspark()



initializing pyspark
pyspark initialized


In [2]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
print(iris.keys())
print(iris['feature_names'])

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [3]:
iris_features = iris['data']
iris_label = iris['target']
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= features + ['target'])
irisDF = spark.createDataFrame(data1)
display(irisDF)

irisPandas = irisDF.toPandas()


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


In [5]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=features, outputCol="features")
dfML = vecAssembler.transform(irisDF)
display(dfML)


,sepal_length,sepal_width,petal_length,petal_width,target,features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]"
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]"
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]"
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]"
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]"
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]"
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]"
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]"
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]"
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]"


In [ ]:
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.sql.functions import expr

CLUSTERS = 3

kmeans = KMeans(k=3, seed = 1)
model = kmeans.fit(dfML)
centroids = model.clusterCenters()
#print(centroids)

predictions = model.transform(dfML) 
#display(predictions)


x = predictions.select('prediction', 'target').collect()
print(x[0])
print('-->', tuple(x[0]))
print(x[0].prediction, x[0]['target'])

# good recipe to convert list of Row objects into list of tuples
print(list(map(tuple, x)))



In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import SQLTransformer
display(irisDF)
sql = "SELECT *, sepal_length/sepal_width as sepal_ratio, petal_length/petal_width as petal_ratio FROM __THIS__"
sqlTransformer = SQLTransformer(statement = sql)
sqlModel = sqlTransformer.transform(irisDF)
display(sqlModel)


,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


,sepal_length,sepal_width,petal_length,petal_width,target,sepal_ratio,petal_ratio
0,5.1,3.5,1.4,0.2,0.0,1.457143,7.000000
1,4.9,3.0,1.4,0.2,0.0,1.633333,7.000000
2,4.7,3.2,1.3,0.2,0.0,1.468750,6.500000
3,4.6,3.1,1.5,0.2,0.0,1.483871,7.500000
4,5.0,3.6,1.4,0.2,0.0,1.388889,7.000000
5,5.4,3.9,1.7,0.4,0.0,1.384615,4.250000
6,4.6,3.4,1.4,0.3,0.0,1.352941,4.666667
7,5.0,3.4,1.5,0.2,0.0,1.470588,7.500000
8,4.4,2.9,1.4,0.2,0.0,1.517241,7.000000
9,4.9,3.1,1.5,0.1,0.0,1.580645,15.000000


In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
display(irisDF.describe())

standardScaler = StandardScaler(inputCol='features', outputCol='scaled_features')
model = standardScaler.fit(dfML)
print(model.mean, model.std)
dfMLScaled = model.transform(dfML)
display(dfML)
display(dfMLScaled)


,summary,sepal_length,sepal_width,petal_length,petal_width,target
0,count,150,150,150,150,150
1,mean,5.843333333333335,3.057333333333334,3.7580000000000027,1.199333333333334,1.0
2,stddev,0.8280661279778637,0.43586628493669793,1.7652982332594662,0.7622376689603467,0.8192319205190406
3,min,4.3,2.0,1.0,0.1,0.0
4,max,7.9,4.4,6.9,2.5,2.0


[5.843333333333333,3.0573333333333315,3.758,1.1993333333333336] [0.8280661279778637,0.4358662849366979,1.7652982332594664,0.7622376689603467]


,sepal_length,sepal_width,petal_length,petal_width,target,features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]"
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]"
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]"
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]"
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]"
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]"
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]"
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]"
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]"
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]"


,sepal_length,sepal_width,petal_length,petal_width,target,features,scaled_features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]","[6.158928408838787, 8.029985619347261, 0.79306..."
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]","[5.9174018045706, 6.882844816583367, 0.7930671..."
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]","[5.675875200302412, 7.341701137688926, 0.73641..."
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]","[5.555111898168318, 7.112272977136146, 0.84971..."
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]","[6.038165106704694, 8.25941377990004, 0.793067..."
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]","[6.52121831524107, 8.947698261558378, 0.963010..."
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]","[5.555111898168318, 7.800557458794483, 0.79306..."
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]","[6.038165106704694, 7.800557458794483, 0.84971..."
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]","[5.313585293900131, 6.653416656030588, 0.79306..."
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]","[5.9174018045706, 7.112272977136146, 0.8497147..."


In [7]:
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import RFormula

Rform = RFormula(formula = 'target~sepal_length + sepal_width + petal_length + petal_width').fit(irisDF).transform(irisDF)
display(Rform)


,sepal_length,sepal_width,petal_length,petal_width,target,features,label
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]",0.0
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]",0.0
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]",0.0
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]",0.0
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]",0.0
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]",0.0
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]",0.0
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]",0.0
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]",0.0
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]",0.0


In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import RFormula

quantile = QuantileDiscretizer(inputCol='sepal_width', outputCol='sepal_width_bucket', numBuckets = 2).fit(irisDF).transform(irisDF)

display(quantile)

# d = {'inputCol':'sepal_length', 'outputCol'='sepal_length_bucket', 'numBuckets' = 2}
# d1 = dict(inputCol='sepal_length', outputCol='sepal_length_bucket', numBuckets = 2)
# quantile1 = QuantileDiscretizer(**d)

quantile1 = QuantileDiscretizer(inputCol='sepal_length', outputCol='sepal_length_bucket', numBuckets = 2)
quantile2 = QuantileDiscretizer(inputCol='sepal_width', outputCol='sepal_width_bucket', numBuckets = 2)
quantile3 = QuantileDiscretizer(inputCol='petal_length', outputCol='petal_length_bucket', numBuckets = 2)
quantile4 = QuantileDiscretizer(inputCol='petal_width', outputCol='petal_width_bucket', numBuckets = 2)


sql = "SELECT *, sepal_length * sepal_length_bucket as sl FROM __THIS__"
sqlTransformer = SQLTransformer(statement = sql)


Rform = RFormula(formula = 'target~sepal_length_bucket + sepal_width_bucket + petal_length_bucket + petal_width_bucket')

stages = [quantile1, quantile2, sqlTransformer, quantile3, quantile4, Rform]

pipeline = Pipeline(stages = stages)

dfPipe = pipeline.fit(irisDF).transform(irisDF)
dfPipe2 = dfPipe.select('label', 'features')
display(dfPipe)

,sepal_length,sepal_width,petal_length,petal_width,target,sepal_width_bucket
0,5.1,3.5,1.4,0.2,0.0,1.0
1,4.9,3.0,1.4,0.2,0.0,1.0
2,4.7,3.2,1.3,0.2,0.0,1.0
3,4.6,3.1,1.5,0.2,0.0,1.0
4,5.0,3.6,1.4,0.2,0.0,1.0
5,5.4,3.9,1.7,0.4,0.0,1.0
6,4.6,3.4,1.4,0.3,0.0,1.0
7,5.0,3.4,1.5,0.2,0.0,1.0
8,4.4,2.9,1.4,0.2,0.0,0.0
9,4.9,3.1,1.5,0.1,0.0,1.0


,sepal_length,sepal_width,petal_length,petal_width,target,sepal_length_bucket,sepal_width_bucket,sl,petal_length_bucket,petal_width_bucket,features,label
0,5.1,3.5,1.4,0.2,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
1,4.9,3.0,1.4,0.2,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
2,4.7,3.2,1.3,0.2,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
3,4.6,3.1,1.5,0.2,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
4,5.0,3.6,1.4,0.2,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
5,5.4,3.9,1.7,0.4,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
6,4.6,3.4,1.4,0.3,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
7,5.0,3.4,1.5,0.2,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0
8,4.4,2.9,1.4,0.2,0.0,0.0,0.0,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 0.0)",0.0
9,4.9,3.1,1.5,0.1,0.0,0.0,1.0,0.0,0.0,0.0,"(0.0, 1.0, 0.0, 0.0)",0.0


In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import PCA
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import RFormula

#    , 'SQL' : (SQLTransformer, {'statement':sql})
#    , 'RFormula' : (RFormula, {'formula':'target~sepal_length + sepal_width + petal_length + petal_width'})
#    , 'quantile' : (QuantileDiscretizer, {'inputOutput':True, 'numBuckets':2})


scalers = {
      'standard' : (StandardScaler,{})
    , 'maxAbs' : (MaxAbsScaler,{})
    , 'minMax' : (MinMaxScaler,{})
    , 'PCA' : (PCA, {'k':2})
}

for name, scalerClass in scalers.items():
    print (name)
    c, p = scalerClass
    p.update({'inputCol':'features', 'outputCol':name+'_features'})
    print (p)
    scaler = c(**p)

    model = scaler.fit(dfML).transform(dfML)
    display(model)


standard
{'inputCol': 'features', 'outputCol': 'standard_features'}


,sepal_length,sepal_width,petal_length,petal_width,target,features,standard_features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]","[6.158928408838787, 8.029985619347261, 0.79306..."
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]","[5.9174018045706, 6.882844816583367, 0.7930671..."
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]","[5.675875200302412, 7.341701137688926, 0.73641..."
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]","[5.555111898168318, 7.112272977136146, 0.84971..."
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]","[6.038165106704694, 8.25941377990004, 0.793067..."
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]","[6.52121831524107, 8.947698261558378, 0.963010..."
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]","[5.555111898168318, 7.800557458794483, 0.79306..."
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]","[6.038165106704694, 7.800557458794483, 0.84971..."
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]","[5.313585293900131, 6.653416656030588, 0.79306..."
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]","[5.9174018045706, 7.112272977136146, 0.8497147..."


maxAbs
{'inputCol': 'features', 'outputCol': 'maxAbs_features'}


,sepal_length,sepal_width,petal_length,petal_width,target,features,maxAbs_features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]","[0.6455696202531644, 0.7954545454545454, 0.202..."
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]","[0.620253164556962, 0.6818181818181818, 0.2028..."
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]","[0.5949367088607594, 0.7272727272727273, 0.188..."
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]","[0.5822784810126581, 0.7045454545454545, 0.217..."
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]","[0.6329113924050632, 0.8181818181818181, 0.202..."
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]","[0.6835443037974683, 0.8863636363636362, 0.246..."
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]","[0.5822784810126581, 0.7727272727272726, 0.202..."
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]","[0.6329113924050632, 0.7727272727272726, 0.217..."
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]","[0.5569620253164557, 0.6590909090909091, 0.202..."
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]","[0.620253164556962, 0.7045454545454545, 0.2173..."


minMax
{'inputCol': 'features', 'outputCol': 'minMax_features'}


,sepal_length,sepal_width,petal_length,petal_width,target,features,minMax_features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]","[0.22222222222222213, 0.6249999999999999, 0.06..."
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]","[0.1666666666666668, 0.41666666666666663, 0.06..."
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]","[0.11111111111111119, 0.5, 0.05084745762711865..."
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]","[0.08333333333333327, 0.4583333333333333, 0.08..."
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]","[0.19444444444444448, 0.6666666666666666, 0.06..."
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]","[0.30555555555555564, 0.7916666666666665, 0.11..."
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]","[0.08333333333333327, 0.5833333333333333, 0.06..."
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]","[0.19444444444444448, 0.5833333333333333, 0.08..."
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]","[0.027777777777777922, 0.3749999999999999, 0.0..."
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]","[0.1666666666666668, 0.4583333333333333, 0.084..."


PCA
{'k': 2, 'inputCol': 'features', 'outputCol': 'PCA_features'}


,sepal_length,sepal_width,petal_length,petal_width,target,features,PCA_features
0,5.1,3.5,1.4,0.2,0.0,"[5.1, 3.5, 1.4, 0.2]","[-2.818239506639473, -5.646349823412762]"
1,4.9,3.0,1.4,0.2,0.0,"[4.9, 3.0, 1.4, 0.2]","[-2.7882234453146832, -5.149951351762869]"
2,4.7,3.2,1.3,0.2,0.0,"[4.7, 3.2, 1.3, 0.2]","[-2.613374563549713, -5.182003150742105]"
3,4.6,3.1,1.5,0.2,0.0,"[4.6, 3.1, 1.5, 0.2]","[-2.757022276967599, -5.008653597575749]"
4,5.0,3.6,1.4,0.2,0.0,"[5.0, 3.6, 1.4, 0.2]","[-2.77364859605448, -5.653707089762587]"
5,5.4,3.9,1.7,0.4,0.0,"[5.4, 3.9, 1.7, 0.4]","[-3.2215054997645174, -6.068283025890586]"
6,4.6,3.4,1.4,0.3,0.0,"[4.6, 3.4, 1.4, 0.3]","[-2.681827381868401, -5.237491192299108]"
7,5.0,3.4,1.5,0.2,0.0,"[5.0, 3.4, 1.5, 0.2]","[-2.8762201594623766, -5.49033753652599]"
8,4.4,2.9,1.4,0.2,0.0,"[4.4, 2.9, 1.4, 0.2]","[-2.615982400828456, -4.74864082264096]"
9,4.9,3.1,1.5,0.1,0.0,"[4.9, 3.1, 1.5, 0.1]","[-2.8296093347880547, -5.213178330953537]"
